# Chained Code Remediation Pipeline
This notebook performs a two-step process:
1. **Scans code to extract SELECT queries, data types, and function models**
2. **Maps and transforms data types using S4 mapping from an external Excel, updating code lines if mapping found**
3. **Transforms each SELECT query using the logic from `Transform-SELECT.ipynb`, replacing the original SELECT block.**

Input notebooks required in the working directory:
- `Scan-Program.ipynb` (for scanning ABAP code)
- `Transform-DE-DT.ipynb` (for data element mapping logic)
- `Transform-SELECT.ipynb` (for SELECT query transformation)
- `Mapping-DE-DT.xlsx` (for mapping data elements to S4 types)

You can adapt the `sample_code` variable to your own ABAP code for processing.

In [ ]:
# Step 1: Import scan logic from Scan-Program.ipynb
# (copy-paste here, or use %run if on local Jupyter)
import re
from cache_loader import PersistentCache

def scan_keywords(code_lines):
    select_queries = []
    data_types = []
    function_names = []
    select_blocks = []  # New: store (start_idx, end_idx, select_text)
    
    in_select = False
    current_select = []
    select_start = None
    for idx, line in enumerate(code_lines):
        l = line.rstrip()
        # SELECT block
        if not in_select and re.search(r'\bSELECT\b', l, re.IGNORECASE):
            in_select = True
            current_select = [l]
            select_start = idx
            if l.strip().endswith(('.', ';')):
                select_queries.append(' '.join(current_select))
                select_blocks.append((select_start, idx, '\n'.join(current_select)))
                in_select = False
                current_select = []
                select_start = None
            continue
        if in_select:
            current_select.append(l)
            if l.strip().endswith(('.', ';')):
                select_queries.append(' '.join(current_select))
                select_blocks.append((select_start, idx, '\n'.join(current_select)))
                in_select = False
                current_select = []
                select_start = None
            continue
        # DATA: TYPE after TYPE keyword
        if re.search(r'\bDATA\b', l, re.IGNORECASE):
            match = re.search(r'TYPE\s+([\w\.\_]+)', l, re.IGNORECASE)
            if match:
                data_types.append(match.group(1))
        # CALL FUNCTION: function name
        if re.search(r'\bCALL FUNCTION\b', l, re.IGNORECASE):
            match = re.search(r'CALL FUNCTION\s+["\']?([\w\_]+)["\']?', l, re.IGNORECASE)
            if match:
                function_names.append(match.group(1))
    return select_queries, data_types, function_names, select_blocks

## Sample ABAP Code
Replace `sample_code` with your own codebase as a list of lines for testing.

In [ ]:
sample_code = [
    'SELECT *',
    '  FROM users',
    '  WHERE id = 1;',
    'DATA lv_name TYPE string.',
    'CALL FUNCTION "BAPI_USER_CREATE".',
    'WRITE lv_name.',
    'SELECT id, name',
    '  FROM customers',
    '  WHERE active = 1',
    '  ORDER BY name.',
    'data: lt_orders TYPE TABLE_OF_ORDERS.',
    'call function "Z_CUSTOM_MODULE".',
    'DATA number TYPE i.',
    'CALL FUNCTION \'RFC_PING\'.'
]

In [ ]:
# Step 2: Run scan to extract SELECTs, Data Types, Functions and SELECT blocks
selects, data_types, functions, select_blocks = scan_keywords(sample_code)
print('SELECT Queries:')
for s in selects:
    print(s)
print('\nDATA TYPEs:')
for d in data_types:
    print(d)
print('\nFUNCTION Names:')
for f in functions:
    print(f)
print('\nSELECT Blocks:')
for b in select_blocks:
    print(f'Lines {b[0]} to {b[1]}:')
    print(b[2])

## Step 3: Import the Data Element Mapping Logic
We use the mapping function from `Transform-DE-DT.ipynb` for S4 mapping.

Make sure `Mapping-DE-DT.xlsx` is present in the working directory.

In [ ]:
import pandas as pd
mapping_file = 'Mapping-DE-DT.xlsx'
df_map = pd.read_excel(mapping_file)

def get_s4_declaration(data_element):
    row = df_map.loc[df_map['CRM'] == data_element]
    if row.empty:
        return None  # Indicate no mapping
    s4 = str(row.iloc[0]['S4']).strip()
    length = row.iloc[0]['LENGTH'] if 'LENGTH' in row.columns else None
    decimal = row.iloc[0]['DECIMAL'] if 'DECIMAL' in row.columns else None
    result = s4
    if pd.notnull(length):
        result += f" LENGTH {int(float(length))}"
    if decimal is not None and pd.notnull(decimal):
        result += f" DECIMALS {int(float(decimal))}"
    return result

## Step 4: Create a Remediation Plan
- For each data type found, get the mapped S4 declaration (if available).
- For each line in code, replace original data type with mapped S4 declaration **only if mapping is found**.
- If no mapping, leave code unchanged.

In [ ]:
# Build mapping dict: {original_type: mapped_declaration}
mapping_results = {}
for dt in set(data_types):
    mapped = get_s4_declaration(dt)
    if mapped:
        mapping_results[dt] = mapped

print('Mapping Results:')
for k, v in mapping_results.items():
    print(f'{k} -> {v}')

## Step 5: Replace Code with Mapped Data Types
- Only change the TYPE clause if mapping is found.

In [ ]:
import copy

remediated_code = copy.deepcopy(sample_code)

for i, line in enumerate(remediated_code):
    if re.search(r'\bDATA\b', line, re.IGNORECASE):
        match = re.search(r'(TYPE\s+)([\w\.\_]+)', line, re.IGNORECASE)
        if match:
            original_type = match.group(2)
            if original_type in mapping_results:
                # Replace only the type part
                mapped_decl = mapping_results[original_type]
                new_line = re.sub(r'(TYPE\s+)([\w\.\_]+)', f"\\1{mapped_decl}", line, count=1, flags=re.IGNORECASE)
                remediated_code[i] = new_line

print('--- Remediated Code after Data Type Mapping ---')
for l in remediated_code:
    print(l)

## Step 6: Transform SELECT Queries using Transform-SELECT.ipynb
- For each SELECT block, use the transformation logic from `Transform-SELECT.ipynb` to get the S4 version.
- Replace the original SELECT block in `remediated_code` with the transformed query.
- If block is multi-line, replace all lines in the block with the single transformed line.

In [ ]:
# Import SELECT transform logic
# Use %run if running interactively, or copy-paste the transform_query_auto function and mapping load logic here
import pandas as pd
import re

# --- BEGIN: Copy/paste from Transform-SELECT.ipynb ---
mapping_url = "https://github.com/srinivashj1/code/raw/main/Select%20Queries%201.xlsx"
mapping_df = pd.read_excel(mapping_url, sheet_name="CRM_S4_TABLE_FIELDS_MAPPING")

def transform_query_with_conditional_join_removal(source_query, mapping_df):
    query = re.sub(r'\s+', ' ', source_query.strip())

    # 1. Detect if SINGLE is present
    single = bool(re.search(r'\bSINGLE\b', query, re.IGNORECASE))

    # 2. Detect FOR ALL ENTRIES
    fae_match = re.search(r'FOR ALL ENTRIES IN (\w+)', query, re.IGNORECASE)
    fae_clause = ''
    if fae_match:
        fae_clause = f'FOR ALL ENTRIES IN {fae_match.group(1).upper()}'

    # 3. Detect UP TO ... ROWS
    up_to_match = re.search(r'UP TO (\w+) ROWS', query, re.IGNORECASE)
    up_to_clause = ''
    if up_to_match:
        up_to_clause = f'UP TO {up_to_match.group(1)} ROWS'

    # 4. Remove INNER JOIN for CRMD_LINK and for tables with Remove Join == 'X'
    join_pattern = r'INNER JOIN (\w+) AS (\w+) ON ([^I]+?)(?=INNER JOIN|\bWHERE\b|\bINTO\b|$)'
    joins = re.findall(join_pattern, query, re.IGNORECASE)
    remove_tables = set()
    for table, alias, _ in joins:
        table_upper = table.upper()
        # Remove CRMD_LINK always
        if table_upper == 'CRMD_LINK':
            remove_tables.add(table_upper)
        mapping_row = mapping_df[mapping_df['CRM_TABNAME'].str.upper() == table_upper]
        if not mapping_row.empty and 'Remove Join' in mapping_row.columns:
            if (mapping_row['Remove Join'] == 'X').any():
                remove_tables.add(table_upper)
    def join_replacer(match):
        table = match.group(1).upper()
        if table in remove_tables:
            return ''
        return match.group(0)
    query = re.sub(join_pattern, join_replacer, query, flags=re.IGNORECASE)

    # 5. Extract fields and their table aliases
    fields = []
    select_fields_match = re.search(r'SELECT (.*?) FROM', query, re.IGNORECASE)
    if select_fields_match:
        fields_str = select_fields_match.group(1).strip()
        if ',' in fields_str:
            fields = [f.strip() for f in fields_str.split(',')]
        else:
            fields = [f.strip() for f in fields_str.split() if f.strip()]
    else:
        fields_match = re.search(r'FIELDS (.*?)(WHERE|INTO|$)', query, re.IGNORECASE)
        if fields_match:
            fields_str = fields_match.group(1).strip().rstrip(',')
            if ',' in fields_str:
                fields = [f.strip() for f in fields_str.split(',')]
            else:
                fields = [f.strip() for f in fields_str.split() if f.strip()]
    fields = [f for f in fields if f.strip().upper() != 'SINGLE']

    # 6. Extract table aliases from FROM/JOIN clauses
    alias_table_map = {}
    for match in re.finditer(r'FROM (\w+)(?: AS (\w+))?', query, re.IGNORECASE):
        table = match.group(1).upper()
        alias = match.group(2).upper() if match.group(2) else table
        alias_table_map[alias] = table
    for match in re.finditer(r'JOIN (\w+)(?: AS (\w+))?', query, re.IGNORECASE):
        table = match.group(1).upper()
        alias = match.group(2).upper() if match.group(2) else table
        alias_table_map[alias] = table

    # 7. Transform fields using mapping with table context
    transformed_fields = []
    for field in fields:
        field_for_map = re.sub(r'@DATA\((.*?)\)', r'\1', field)
        field_for_map = field_for_map.lstrip('@')
        if '~' in field_for_map:
            alias, fname = field_for_map.split('~', 1)
            crm_table = alias_table_map.get(alias.upper(), alias.upper())
        else:
            fname = field_for_map
            crm_table = list(alias_table_map.values())[0] if alias_table_map else ''
        fname = fname.strip().upper()
        mapping_row = mapping_df[(mapping_df['CRM_TABNAME'].str.upper() == crm_table) &
                                 (mapping_df['CRM_FIELDNAME'].str.upper() == fname)]
        if not mapping_row.empty:
            s4_field = mapping_row.iloc[0]['S4_FIELDNAME']
        else:
            s4_field = fname
        transformed_fields.append(f'{s4_field} AS {fname}')

    # 8. Extract INTO clause
    into_clause = ''
    into_match = re.search(r'INTO\s+(TABLE\s+)?((@?DATA\([^)]+\))|(@?\w+))', query, re.IGNORECASE)
    if into_match:
        into_var = into_match.group(2)
        if into_match.group(1):
            into_clause = f'INTO TABLE {into_var}'
        else:
            into_clause = f'INTO {into_var}'

    # 9. Extract WHERE clause, map fields with table context
    where_clause = ''
    where_match = re.search(r'WHERE (.*?)(FOR ALL ENTRIES|INTO|UP TO|$)', query, re.IGNORECASE)
    if where_match:
        where_cond = where_match.group(1).strip()
        def replace_tilde(m):
            alias = m.group(1).upper()
            fname = m.group(2).upper()
            crm_table = alias_table_map.get(alias, alias)
            mapping_row = mapping_df[(mapping_df['CRM_TABNAME'].str.upper() == crm_table) &
                                     (mapping_df['CRM_FIELDNAME'].str.upper() == fname)]
            if not mapping_row.empty:
                s4_field = mapping_row.iloc[0]['S4_FIELDNAME']
            else:
                s4_field = fname
            return s4_field
        where_cond = re.sub(r'(\w+)~(\w+)', replace_tilde, where_cond)
        where_clause = f'WHERE {where_cond}'

    # 10. Map main table name
    main_table_alias = list(alias_table_map.keys())[0] if alias_table_map else ''
    main_crm_table = alias_table_map.get(main_table_alias, '')
    mapping_row = mapping_df[mapping_df['CRM_TABNAME'].str.upper() == main_crm_table]
    s4_table = mapping_row.iloc[0]['S4_TABNAME'] if not mapping_row.empty else main_crm_table

    # 11. Compose the final query
    parts = []
    if single:
        parts.append('SELECT SINGLE')
    else:
        parts.append('SELECT')
    parts.append(', '.join(transformed_fields))
    if s4_table:
        parts.append(f'FROM {s4_table}')
    if into_clause:
        parts.append(into_clause)
    if fae_clause:
        parts.append(fae_clause)
    if up_to_clause:
        parts.append(up_to_clause)
    if where_clause:
        parts.append(where_clause)

    transformed_query = ' '.join(parts)
    return transformed_query

def transform_query_with_partner_handling(source_query, mapping_df):
    # (For brevity, not included in this snippet, see previous code cell for full function)
    pass

def transform_query_auto(source_query, mapping_df):
    q = re.sub(r'\s+', ' ', source_query.strip().upper())
    partner_join = bool(re.search(r'JOIN\s+CRMD_PARTNER', q))
    if partner_join:
        return transform_query_with_partner_handling(source_query, mapping_df)
    else:
        return transform_query_with_conditional_join_removal(source_query, mapping_df)
# --- END: Copy/paste from Transform-SELECT.ipynb ---

In [ ]:
# Replace SELECT blocks in remediated_code with transformed queries
remediated_code_with_selects = copy.deepcopy(remediated_code)

for (start, end, select_block) in select_blocks:
    original_query = select_block
    transformed_query = transform_query_auto(original_query, mapping_df)
    # If original query was multi-line, replace all lines with a single transformed line
    remediated_code_with_selects[start:end+1] = [transformed_query]

print('--- Remediated Code after SELECT Query Transformation ---')
for l in remediated_code_with_selects:
    print(l)

## Summary
- Extracted SELECT queries, Data Types, and Function Names
- Mapped and replaced Data Types in code using the mapping sheet, only where mapping found
- Transformed SELECT queries using S4 mapping and replaced the original blocks in the code
- Output is a fully remediated code list with data type and SELECT query modernization